# Chapter 8
# Data Aggregation and Group Operations

Categorizing a dataset and applying a function to each group, whether an aggregationor transformation, is often a critical component of a data analysis workflow. After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly pivot tables for reporting or visualization purposes. pandas provides a flexible groupby interface, enabling you to slice, dice, and summarize datasets in a natural way.

One reason for the popularity of relational databases and SQL (which stands for “structured query language”) is the ease with which data can be joined, filtered, transformed, and aggregated. However, query languages like SQL are somewhat constrained in the kinds of group operations that can be performed. As you will see, with the expressiveness of Python and pandas, we can perform quite complex group operations by utilizing any function that accepts a pandas object or NumPy array.

## 8.1 GroupBy Mechanics

Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations. In the first stage of the process, data contained in a pandas object, whether a Series, Data‐
Frame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object. 

For example, a DataFrame can be grouped on its rows *(axis=0)* or its columns *(axis=1)*. Once this is done, a function is applied to each group, producing a new value. Finally, the results of all those function applications are combined into a result object. The form of the resulting object will usually depend on what’s being done to the data. See Figure 8-1 for a
mockup of a simple group aggregation.

<img src="Figure 8.1.png" width="600px">
<br>
<center>Figure 8.1: Illustration of a group aggregation</center>

Each grouping key can take many forms, and the keys do not have to be all of the same type:<br>
• A list or array of values that is the same length as the axis being grouped <br>
• A value indicating a column name in a DataFrame <br>
• A dict or Series giving a correspondence between the values on the axis being grouped and the group names <br>
• A function to be invoked on the axis index or the individual labels in the index <br>

Note that the latter three methods are shortcuts for producing an array of values to be used to split up the object.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,2.001162,-0.526941
1,a,two,1.417388,0.333847
2,b,one,-0.899198,-0.782425
3,b,two,-0.660799,1.631486
4,a,one,0.658426,0.130588


Suppose you wanted to compute the mean of the data1 column using the labels from *key1*. There are a number of ways to do this. One is to access data1 and call `groupby` with the column (a Series) at key1:

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

This grouped variable is now a *GroupBy* object. It has not actually computed anything yet except for some intermediate data about the group key df['key1']. The idea is that this object has all of the information needed to then apply some operation to
each of the groups. For example, to compute group means we can call the GroupBy’s `mean` method:

In [4]:
grouped.mean()

key1
a    1.358992
b   -0.779999
Name: data1, dtype: float64

The important thing here is that the data (a Series) has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the key1 column.

The result index has the name 'key1' because the DataFrame column df['key1'] did.

If instead we had passed multiple arrays as a list, we’d get something different:

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.329794
      two     1.417388
b     one    -0.899198
      two    -0.660799
Name: data1, dtype: float64

Here we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed:

In [6]:
means.unstack()

key2,one,two
key1,,
a,1.329794,1.417388
b,-0.899198,-0.660799


In this example, the group keys are all Series, though they could be any arrays of the right length:

In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    1.417388
            2006   -0.899198
Ohio        2005    0.670181
            2006    0.658426
Name: data1, dtype: float64

Frequently the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names (whether those are strings, numbers, or other Python objects) as the group keys:


In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.358992,-0.020836
b,-0.779999,0.424531


In [9]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.329794 -0.198177
     two   1.417388  0.333847
b    one  -0.899198 -0.782425
     two  -0.660799  1.631486

You may have noticed in the first case df.groupby('key1').mean() that there is no key2 column in the result. Because df['key2'] is not numeric data, it is said to be a *nuisance column*, which is therefore excluded from the result. By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset.

Regardless of the objective in using groupby, a generally useful GroupBy method is size, which returns a Series containing group sizes:

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

Take note that any missing values in a group key will be **excluded** from the result.

### 8.1.1 Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. Consider the following:

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  2.001162 -0.526941
1    a  two  1.417388  0.333847
4    a  one  0.658426  0.130588
b
  key1 key2     data1     data2
2    b  one -0.899198 -0.782425
3    b  two -0.660799  1.631486


In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  2.001162 -0.526941
4    a  one  0.658426  0.130588
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.417388  0.333847
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.899198 -0.782425
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.660799  1.631486


You can choose to do whatever you want with the pieces of data. A recipe you may find useful is computing a dict of the data pieces as a one-liner:

In [13]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.899198,-0.782425
3,b,two,-0.660799,1.631486


By default `groupby` groups on *axis=0*, but you can group on any of the other axes. For example, we could group the columns of our example df here by dtype like so:

In [14]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)

We can print out the groups like so:

In [16]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  2.001162 -0.526941
1  1.417388  0.333847
2 -0.899198 -0.782425
3 -0.660799  1.631486
4  0.658426  0.130588
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 8.1.2 Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:

    df.groupby('key1')['data1']
    df.groupby('key1')[['data2']]

are syntactic sugar for:

    df['data1'].groupby(df['key1'])
    df[['data2']].groupby(df['key1'])

Especially for large datasets, it may be desirable to aggregate only a few columns. For example, in the preceding dataset, to compute means for just the data2 column and get the result as a DataFrame, we could write:

In [17]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.198177
     two   0.333847
b    one  -0.782425
     two   1.631486

The object returned by this indexing operation is a grouped DataFrame if a list or array is passed or a grouped Series if only a single column name is passed as a scalar:

In [18]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [19]:
s_grouped.mean()

key1  key2
a     one    -0.198177
      two     0.333847
b     one    -0.782425
      two     1.631486
Name: data2, dtype: float64

### 8.1.3 Grouping with Dicts and Series

Grouping information may exist in a form other than an array. Let’s consider another example DataFrame:

In [20]:
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.142220,0.767782,1.626964,-2.028477,0.369278
Steve,-0.432719,0.538126,0.134150,0.099621,-1.164477
Wes,-1.515310,NaN,NaN,0.146259,1.992856
Jim,-0.914307,2.087667,0.170395,-0.985270,1.259740
Travis,0.181828,-0.453876,0.394501,-0.585641,-1.085488


Now, suppose we have a group correspondence for the columns and want to sum together the columns by group:

In [21]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}

Now, we could construct an array from this dict to pass to groupby, but instead we can just pass the dict (we included the key 'f' to highlight that unused grouping keys are OK):

In [22]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.401513,1.279280
Steve,0.233771,-1.059070
Wes,0.146259,0.477546
Jim,-0.814875,2.433100
Travis,-0.191140,-1.357537


The same functionality holds for Series, which can be viewed as a fixed-size mapping:

In [23]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 8.1.4 Grouping with Functions

Using Python functions is a more generic way of defining a group mapping compared with a dict or Series. Any function passed as a group key will be called once per index value, with the return values being used as the group names. More concretely, con‐
sider the example DataFrame from the previous section, which has people’s first names as index values. 

Suppose we wanted to group by the length of the names; while we could compute an array of string lengths, it’s simpler to just pass the `len` function:

In [25]:
people.groupby(len).sum()

,a,b,c,d,e
3,-2.287397,2.855449,1.797359,-2.867488,3.621874
5,-0.432719,0.538126,0.134150,0.099621,-1.164477
6,0.181828,-0.453876,0.394501,-0.585641,-1.085488


Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

In [27]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.515310  0.767782  1.626964 -2.028477  0.369278
  two -0.914307  2.087667  0.170395 -0.985270  1.259740
5 one -0.432719  0.538126  0.134150  0.099621 -1.164477
6 two  0.181828 -0.453876  0.394501 -0.585641 -1.085488

### 8.1.5 Grouping by Index Levels

A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index. Let’s look at an example:

In [28]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.037356  1.606446  1.180057 -0.615356 -0.557396
1      0.975130 -1.209816  1.282088 -1.299433  0.109984
2     -1.692421 -0.519736  0.980424  1.512416 -0.126901
3     -2.044902  0.971429  0.203767  0.556985  0.648471

To group by level, pass the level number or name using the `level` keyword:

In [29]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 8.2 Data Aggregation

Aggregations refer to any data transformation that produces scalar values from arrays. The preceding examples have used several of them, including `mean`, `count`, `min`, and `sum`. We may wonder what is going on when we invoke `mean()` on a GroupBy object. Many common aggregations, such as those found in Table 8.1, have optimized implementations. However, we are not limited to only this set of methods.

<br><center>Table 8.1: Optimized groupby methods</center>
<img src="Table 8.1.png" width="500px">

We can use aggregations of our own devising and additionally call any method that is also defined on the grouped object. For example, we might recall that `quantile` computes sample quantiles of a Series or a DataFrame’s columns.

While `quantile` is not explicitly implemented for GroupBy, it is a Series method and thus available for use. Internally, GroupBy efficiently slices up the Series, calls `piece.quantile(0.9)` for each piece, and then assembles those results together into the result object:

In [30]:
df

,key1,key2,data1,data2
0,a,one,2.001162,-0.526941
1,a,two,1.417388,0.333847
2,b,one,-0.899198,-0.782425
3,b,two,-0.660799,1.631486
4,a,one,0.658426,0.130588


In [31]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.884407
b   -0.684639
Name: data1, dtype: float64

In [34]:
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


To use your own aggregation functions, pass any function that aggregates an array to the `aggregate` or `agg` method:

In [32]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.342737,0.860789
b,0.238399,2.413911


We may notice that some methods like describe also work, even though they are not aggregations, strictly speaking:

In [ ]:
grouped.describe()

### 8.2.1 Column-Wise and Multiple Function Application

Let’s go to the tipping dataset. After loading it with `read_csv`, we add a tipping percentage column *tip_pct*:

In [35]:
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [36]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:10]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240
6,8.77,2.00,Male,No,Sun,Dinner,2,0.228050
7,26.88,3.12,Male,No,Sun,Dinner,4,0.116071
8,15.04,1.96,Male,No,Sun,Dinner,2,0.130319
9,14.78,3.23,Male,No,Sun,Dinner,2,0.218539


As we’ve already seen, aggregating a Series or all of the columns of a DataFrame is a matter of using `aggregate` with the desired function or calling a method like `mean` or `std`. However, we may want to aggregate using a different function depending on the column, or multiple functions at once. Fortunately, this is possible to do, which we’ll illustrate through a number of examples. First, we’ll group the *tips* by day and *smoker*:

In [37]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions:

In [38]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Here we passed a list of aggregation functions to `agg` to evaluate indepedently on the data groups.

We don’t need to accept the names that GroupBy gives to the columns; notably, `lambda` functions have the name `'<lambda>'`, which makes them hard to identify. Thus, if we pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names (we can think of a list of 2-tuples as an ordered mapping):

In [39]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame we have more options, as we can specify a list of functions to apply to all of the columns or different functions per column. To start, suppose we wanted to compute the same three statistics for the *tip_pct* and *total_bill*
columns:

In [40]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

As we can see, the resulting DataFrame has hierarchical columns, the same as we would get aggregating each column separately and using `concat` to glue the results together using the column names as the `keys` argument:

In [41]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

As before, a list of tuples with custom names can be passed:

In [42]:
ftuples = [('Average', 'mean'), ('Variance', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

Now, suppose we wanted to apply potentially different functions to one or more of the columns. To do this, pass a dict to `agg` that contains a mapping of column names to any of the function specifications listed so far:

In [45]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [43]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [46]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

A DataFrame will have hierarchical columns only if multiple functions are applied to at least one column.

### 8.2.2 Returning Aggregated Data Without Row Indexes

In all of the examples up until now, the aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations. Since this isn’t always desirable, we can disable this behavior in most cases by passing
`as_index=False` to `groupby`:

In [47]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


Of course, it’s always possible to obtain the result in this format by calling `reset_index` on the result. Using the `as_index=False` method avoids some unnecessary computations.

## 8.3 Apply: General split-apply-combine

The most general-purpose GroupBy method is apply, which is the subject of the rest of this section. As illustrated in Figure 8.1 previously, apply splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together.

<img src="Figure 8.1.png" width="600px">
<br>
<center>Figure 8.1: Illustration of a group aggregation</center>

Returning to the tipping dataset from before, suppose we wanted to select the top five *tip_pct* values by group. First, write a function that selects the rows with the largest values in a particular column:

In [ ]:
tips

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

Now, if we group by *smoker*, say, and call apply with this function, we get the following:

In [ ]:
tips.groupby('smoker').apply(top)

What has happened here? The top function is called on each row group from the DataFrame, and then the results are glued together using `pandas.concat`, labeling the pieces with the group names. The result therefore has a hierarchical index whose
inner level contains index values from the original DataFrame.

If you pass a function to apply that takes other arguments or keywords, you can pass these after the function:

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

We may recall that we earlier called `describe` on a GroupBy object:

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

In [ ]:
result.unstack()

In [ ]:
result.unstack('smoker')

Inside GroupBy, when we invoke a method like `describe`, it is actually just a shortcut for:

In [ ]:
f = lambda x: x.describe()
grouped.apply(f)

### 8.3.1 Suppressing the Group Keys

In the preceding examples, we see that the resulting object has a hierarchical index formed from the group keys along with the indexes of each piece of the original object. We can disable this by passing `group_keys=False` to `groupby`:

In [50]:
tips.groupby('smoker', group_keys=False).apply(top)

NameError: name 'top' is not defined

### 8.3.2 Quantile and Bucket Analysis

As we may recall from Chapter 6, pandas has some tools, in particular `cut` and `qcut`, for slicing data up into buckets with bins of your choosing or by sample quantiles. Combining these functions with `groupby` makes it convenient to perform bucket or
quantile analysis on a dataset. Consider a simple random dataset and an equal-length bucket categorization using `cut`:

In [ ]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

The `Categorical` object returned by cut can be passed directly to groupby. So we could compute a set of statistics for the *data2* column like so:

In [49]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

NameError: name 'frame' is not defined

These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use `qcut`. We’ll pass `labels=False` to just get quantile numbers:

In [48]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

NameError: name 'frame' is not defined

### 8.3.3 Example: Filling Missing Values with Group-Specific Values

When cleaning up missing data, in some cases we will replace data observations using `dropna`, but in others we may want to impute (fill in) the null (NA) values using a fixed value or some value derived from the data. `fillna` is the right tool to
use; for example, here we fill in NA values with the mean:

In [ ]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

In [ ]:
s.fillna(s.mean())

Suppose we need the fill value to vary by group. One way to do this is to group the data and use `apply` with a function that calls `fillna` on each data chunk. Here is some sample data on US states divided into eastern and western regions:

In [51]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.882838
New York     -0.572482
Vermont       0.695805
Florida       1.703174
Oregon        0.015646
Nevada       -1.012109
California    0.103557
Idaho        -0.895439
dtype: float64

Note that the syntax ['East'] * 4 produces a list containing four copies of the elements in ['East']. Adding lists together concatenates them. Let’s set some values in the data to be missing:

In [52]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.882838
New York     -0.572482
Vermont            NaN
Florida       1.703174
Oregon        0.015646
Nevada             NaN
California    0.103557
Idaho              NaN
dtype: float64

In [ ]:
data.groupby(group_key).mean()

We can fill the NA values using the group means like so:

In [53]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.882838
New York     -0.572482
Vermont       0.082618
Florida       1.703174
Oregon        0.015646
Nevada        0.059601
California    0.103557
Idaho         0.059601
dtype: float64

In another case, we might have predefined fill values in our code that vary by group. Since the groups have a name attribute set internally, we can use that:

In [54]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.882838
New York     -0.572482
Vermont       0.500000
Florida       1.703174
Oregon        0.015646
Nevada       -1.000000
California    0.103557
Idaho        -1.000000
dtype: float64

## 8.4 Pivot Tables and Cross-Tabulation

### 8.4.1 Pivot Tables

A *pivot table* is a data summarization tool frequently found in spreadsheet programs and other data analysis software. It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns. Pivot tables in Python with pandas are made possible through the `groupby` facility described in this chapter combined with reshape operations utilizing hierarchical indexing. DataFrame has a `pivot_table` method, and there is also a top-level `pandas.pivot_table` function. In addition to providing a convenience interface to groupby, `pivot_table` can add partial totals, also known as *margins*.

Returning to the tipping dataset, suppose you wanted to compute a table of group means (the default `pivot_table` aggregation type) arranged by *day* and *smoker* on the rows:

In [55]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

This could have been produced with `groupby` directly. Now, suppose we want to aggregate only *tip_pct* and *size*, and additionally group by *time*. We’ll put *smoker* in the table columns and day in the rows:

In [57]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [56]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

We could augment this table to include partial totals by passing `margins=True`. This has the effect of adding All row and column labels, with corresponding values being the group statistics for all the data within a single tier:

In [58]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

Here, the *All* values are means without taking into account smoker versus nonsmoker (the All columns) or any of the two levels of grouping on the rows (the All row).

To use a different aggregation function, pass it to `aggfunc`. For example, `count` or `len` will give you a cross-tabulation (count or frequency) of group sizes:

In [59]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [60]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc='count', margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

If some combinations are empty (or otherwise NA), we may wish to pass a `fill_value`:

In [61]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

See Table 8.2 for a summary of pivot_table methods.

<br><center>Table 8.2: *pivot_table* options</center>
<img src="Table 8.2.png" width="800px">

### 8.4.2 Cross-Tabulations: Crosstab

A cross-tabulation (or crosstab for short) is a special case of a pivot table that computes group frequencies. Here is an example:

In [62]:
data1 = pd.DataFrame({'Sample': range(10), 'Nationality': ['USA', 'Japan', 'USA', 'Japan', 'Japan', 'Japan', 'USA',
                                                            'USA', 'Japan', 'USA'], 
                      'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed',
                                     'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed',]})
data1

,Sample,Nationality,Handedness
0,0,USA,Right-handed
1,1,Japan,Left-handed
2,2,USA,Right-handed
3,3,Japan,Right-handed
4,4,Japan,Left-handed
5,5,Japan,Right-handed
6,6,USA,Right-handed
7,7,USA,Left-handed
8,8,Japan,Right-handed
9,9,USA,Right-handed


As part of some survey analysis, we might want to summarize this data by nationality and handedness. We could use `pivot_table` to do this, but the `pandas.crosstab` function can be more convenient:

In [67]:
frame=data1.groupby(['Nationality','Handedness']).agg('count')
frame

Sample
Nationality Handedness          
Japan       Left-handed        2
            Right-handed       3
USA         Left-handed        1
            Right-handed       4

In [63]:
pd.crosstab(data1.Nationality, data1.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


The first two arguments to `crosstab` can each either be an array or Series or a list of arrays. As in the tips data:

In [68]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244